In [1]:
# Sample intents for the grocery bot
intents = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hello", "Hi", "Hey", "Is anyone there?", "Good day"],
            "responses": ["Hello! Welcome to Grocery shop X:) How can I assist you with your grocery needs today?", "Hi there!Welcome to Grocery Shop X:) Looking to order some groceries?", "Hello!Welcome to Grocery Shop X:) What groceries would you like to order today?"]
        },
        {
            "tag": "order_groceries",
            "patterns": ["I need to order some groceries", "Can I place a grocery order?", "I want to buy some items", "I need to purchase groceries", "Order groceries", "food", "what is available", "i need something"],
            "responses": ["Sure! What items would you like to order?", "Please tell me the groceries you would like to buy.", "Let me know the items and quantities, and I'll prepare your order."]
        },
        {
            "tag": "takeaway_option",
            "patterns": ["Do you offer take-away?", "Can I pick up my order?", "Is take-away available?", "Can I collect the order myself?", "Is there a pick-up option?"],
            "responses": ["Yes, take-away is available. You can pick up your order from our store once it's ready.", "We offer a take-away option. Please let us know when you’d like to collect it."]
        },
        {
            "tag": "checkout_order",
            "patterns": ["I'm ready to checkout", "Complete my order", "I want to finish my order", "Finalize the order", "I'm done with my order"],
            "responses": ["Great! I'll summarize your order and save it in a PDF format. One moment, please!"]
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "Goodbye", "See you later", "I'm done", "Thanks, that's all"],
            "responses": ["Thank you for visiting! Have a great day!", "Goodbye! Feel free to come back anytime you need groceries."]
        }
    ]
}


In [2]:
!pip install nltk


In [3]:
# Tokenization, stemming, and bag of words
#!pip install nltk
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
import numpy as np
def tokenize(sentence):
    return nltk.word_tokenize(sentence)#which converts a sentence (a string of words) into a list of words (tokens).

def stem(word):
    return stemmer.stem(word.lower())#It removes suffixes to find the root form of a word.

def bag_of_words(tokenized_sentence, words):
    sentence_words = [stem(word) for word in tokenized_sentence]
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1
    return bag


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Prepare training data
all_words = []
tags = []
xy = []
for intent in intents["intents"]:
    tag = intent["tag"]
    tags.append(tag)
    for pattern in intent["patterns"]:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ["?", "!", ".", ","]
all_words = sorted(set(stem(w) for w in all_words if w not in ignore_words))
tags = sorted(set(tags))

X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

#convert list to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

In [5]:
import torch
import torch.nn as nn #ontains the building blocks for creating neural networks.


In [6]:
# Hyperparameters
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
batch_size = 8
learning_rate = 0.001
num_epochs = 1000

# model is a 3-layer neural network
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        return out

In [7]:
# Training
model = NeuralNet(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()#is used as the loss function for multi-class classification
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    outputs = model(torch.from_numpy(X_train).float())
    loss = criterion(outputs, torch.from_numpy(y_train).long())
    optimizer.zero_grad()#Resets gradients for all model parameters before backpropagation to avoid accumulating gradients.
    loss.backward()#Calculates gradients of the loss with respect to each model parameter.
    optimizer.step()#Updates the model parameters based on the calculated gradients.

# Function to predict the tag
def predict_class(sentence):
    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = torch.from_numpy(X).float()
    output = model(X)
    _, predicted = torch.max(output, dim=0)
    tag = tags[predicted.item()]
    return tag

In [8]:
predict_class("whats available")

'order_groceries'

In [9]:

import csv

# List of grocery items and their costs
grocery_items = [
    ("milk", 1.50),
    ("bread", 2.00),
    ("eggs", 3.00),
    ("cheese", 4.50),
    ("chicken", 8.00),
    ("beef", 10.00),
    ("pork", 7.00),
    ("fish", 6.00),
    ("rice", 1.00),
    ("pasta", 1.50),
    ("flour", 0.75),
    ("sugar", 1.20),
    ("salt", 0.50),
    ("pepper", 2.50),
    ("butter", 3.00),
    ("olive oil", 5.00),
    ("vegetable oil", 4.00),
    ("vinegar", 1.00),
    ("ketchup", 2.00),
    ("mustard", 1.50),
    ("mayonnaise", 3.00),
    ("yogurt", 1.50),
    ("apples", 0.60),
    ("bananas", 0.50),
    ("oranges", 0.80),
    ("grapes", 2.50),
    ("strawberries", 3.50),
    ("blueberries", 4.00),
    ("raspberries", 5.00),
    ("watermelon", 4.50),
    ("cantaloupe", 3.00),
    ("pineapple", 2.00),
    ("lemon", 0.50),
    ("lime", 0.30),
    ("avocado", 1.00),
    ("nuts", 8.00),
    ("almonds", 9.00),
    ("walnuts", 10.00),
    ("peanuts", 3.00),
    ("cashews", 12.00),
    ("dried fruits", 5.00),
    ("raisins", 3.50),
    ("apricots", 6.00),
    ("granola", 4.00),
    ("cereal", 3.00),
    ("oats", 2.00),
    ("coffee", 8.00),
    ("tea", 3.00),
    ("chocolate", 2.50),
    ("chips", 2.00),
    ("crackers", 2.50),
    ("cookies", 3.00),
    ("candy", 1.00),
    ("ice cream", 4.00),
    ("frozen vegetables", 3.00),
    ("frozen pizza", 5.00),
    ("frozen meals", 7.00),
    ("canned beans", 1.00),
    ("canned tomatoes", 1.50),
    ("canned soup", 2.00),
    ("broth", 1.50),
    ("stock", 2.00),
    ("peanut butter", 3.00),
    ("jelly", 2.00),
    ("pickles", 1.50),
    ("olives", 2.00),
    ("herbs", 1.00),
    ("spices", 2.00),
    ("baking powder", 1.50),
    ("baking soda", 1.00),
    ("yeast", 1.50),
    ("tortillas", 2.50),
    ("pita bread", 2.00),
    ("bagels", 3.00),
    ("English muffins", 3.00),
    ("cream cheese", 2.50),
    ("feta cheese", 3.50),
    ("sour cream", 2.00),
    ("fresh herbs", 1.00),
    ("seasoning mix", 2.00),
    ("hot sauce", 2.50),
    ("soy sauce", 1.50),
    ("teriyaki sauce", 2.00),
    ("barbecue sauce", 2.50),
    ("salad dressing", 3.00),
    ("marinara sauce", 2.00),
    ("curry paste", 2.50),
    ("instant noodles", 0.50),
    ("rice cakes", 1.50),
    ("energy bars", 2.50),
    ("protein powder", 20.00),
    ("vitamin supplements", 15.00),
    ("pet food", 10.00),
    ("cleaning supplies", 5.00),
    ("paper towels", 3.00),
    ("toilet paper", 5.00),
    ("dish soap", 2.00),
    ("laundry detergent", 8.00),
    ("trash bags", 4.00),
]

# Path to save the CSV file
csv_file_path = 'grocery_items.csv'  # this is for google colab for notebook we need to give specific path

# Write to the CSV file
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Item", "Cost"])  # Header
    writer.writerows(grocery_items)

print(f"Grocery items saved to {csv_file_path}")


Grocery items saved to grocery_items.csv


In [10]:
!pip install fpdf
#It's simple to use and allows you to create basic PDFs with text, images, and formatting

In [ ]:

"""
Steps to Fix the Issue
Get the Correct File ID: After copying the PDF to Google Drive, we should ensure we're using the correct file ID for making the file shareable.

Update the Sharing Logic: Instead of trying to derive the file ID from the file name, we'll use the Google Drive API to retrieve the file's metadata after it's uploaded.
"""
from fpdf import FPDF
from PIL import Image
import os
import random
import shutil
from google.colab import files
from google.colab import drive
import pandas as pd
import numpy as np
# this is important for google drive link, instead of credential json file use this
"""
In Google Colab, from google.colab import auth followed by auth.authenticate_user() is sufficient for authentication with Google services because Colab offers built-in authentication integration with Google APIs, making it more convenient than downloading and managing a service account JSON file.
When you run auth.authenticate_user(), Google Colab will prompt you to log in with your Google account (if not already logged in) and authorize Colab to access your Google Drive or other Google services on your behalf.
"""
from google.colab import auth
auth.authenticate_user()
"""
This approach provides access permissions for Google resources, such as Google Drive, Sheets, or BigQuery, without needing user consent each time.
"""
from google.oauth2 import service_account#This module provides tools for service account authentication
from googleapiclient.discovery import build#This module allows you to create a service object that acts as an interface to Google APIs, such as Google Drive, Sheets, Gmail, etc.
# for emails
import smtplib#This module defines an SMTP (Simple Mail Transfer Protocol) client session object that can be used to send email using an SMTP server
from email.mime.multipart import MIMEMultipart#used to create a multipart email message. A multipart email can contain different types of content, such as plain text, HTML, and attachments.
from email.mime.text import MIMEText# module and is used to create email messages that contain plain text or HTML content.


grand_total = 0
def find_item_cost(item_name, quantity):
    global grand_total
    grocery_data = pd.read_csv(csv_file_path)
    item_details = grocery_data[grocery_data['Item'] == item_name]

    if not item_details.empty:
        cost = float(item_details['Cost'].values[0])
        total_cost = quantity * cost
        grand_total += total_cost  # Add the item's total cost to the grand total
        print(f"{quantity} {item_name}(s) cost {total_cost} rupees.")
        print(f"\nYour total amount for all items is {grand_total} rupees.")
    else:
        print(f"Item '{item_name}' not found in the list.")

#  Get the file ID and make the PDF shareable, file_id represents the unique identifier for a file stored in Google Drive. Each file has a distinct file ID that the API uses to identify and access that particular file.
def make_file_shareable(file_name):
    service = build('drive', 'v3')# version3 of google drive API

    # Search for the file in Google Drive
    results = service.files().list(q=f"name='{file_name}'", spaces='drive', fields='files(id)').execute()
    items = results.get('files', [])

    if not items:
        print(f"No file found with name: {file_name}")
        return None

    file_id = items[0]['id']  # Get the file ID

    # Create permission
    permission = {
        'type': 'anyone',
        'role': 'reader'
    }

    # Add the permission to the file
    service.permissions().create(
        fileId=file_id,
        body=permission
    ).execute()

    return f"https://drive.google.com/file/d/{file_id}/view?usp=sharing"

def send_email(to_email, subject, body):
    # Set up the server (Gmail in this example)
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587
    sender_email = 'mahendarbs1999@gmail.com'
    sender_password = 'gayb byat shbc cihn'  # Use App Password

    # Create the email
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = to_email
    msg['Subject'] = subject

    # Attach the body to the email
    msg.attach(MIMEText(body, 'plain'))

    # Create the server connection and send the email
    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()  # Enable security
        server.login(sender_email, sender_password)  # Log in to your account
        server.send_message(msg)
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {str(e)}")
    finally:
        server.quit()  # Close the connection

#main chat function
def chat():
  bot_name = "Cyra🤖"
  print(f"Hi! I am {bot_name}. Start chatting with me (type 'quit' to stop)!")
  while True:
    sentence = input("You: ")
    if sentence.lower() == "quit":
        break
    tag = predict_class(sentence)
    responses = next((intent["responses"] for intent in intents["intents"] if intent["tag"] == tag), ["Sorry, I didn’t understand that."])
    selected_response = random.choice(responses)
    print(f"{bot_name}: {selected_response}")

    if tag == "order_groceries":
      print(f"{bot_name}: {grocery_items}")
      items = []
      quantities = []
      while True:
            item = input(f"{bot_name}:Which item would you like to add to your order? order one by one ")
            items.append(item)
            quantity = float(input(f"{bot_name}:How much {item} would you like? (e.g., 1, 2, 0.5): "))
            quantities.append(quantity)
            units = input(f"{bot_name}:Specify units (e.g., kg, pounds, grams, liters, packets, etc.): ")
            costing = find_item_cost(item, quantity)
            print(costing)
            continue_order = input(f"{bot_name}:Would you like to order another item? (yes/no): ").strip().lower()
            if continue_order != "yes":
                break
      print(f"{bot_name}:item is {items},{quantities}")
      phone_number = input(f"{bot_name}:give your phone number")
      adress = input(f"{bot_name}:give your adress example: my name is john  and i am living opposite petrol bunk gubbi")
      pincode = input(f"{bot_name}:give your pincode")
      if pincode != "572223" :
         print(f"{bot_name}:We are not taking orders from this pincode , Sorry for unavailable")
      else:
        email = input(f"{bot_name}:give me your email")
        print(costing)
        payment = input(f"{bot_name}:this is our number 9164453597 and type yes if you done and take screenshot of your payment with transaction ID and upload below option ")
        if payment != "yes":
                break
        else:
            #  Mount Google Drive
          drive.mount('/content/drive')

          # Upload the image
          uploaded = files.upload()

          # Create PDF
          pdf = FPDF()
          pdf.add_page()

          # Add text data
          pdf.set_font("Arial", size=12)
          pdf.cell(200, 10, txt="Order Summary:", ln=True)
          for item, qty in zip(items, quantities):
              pdf.cell(20, 10, txt=f"{item}: {qty}", ln=True)

          # Add other details to PDF
          pdf.cell(20, 10, txt=f"Phone Number: {phone_number}", ln=True)
          pdf.cell(20, 10, txt=f"Address: {adress}", ln=True)
          pdf.cell(20, 10, txt=f"Email: {email}", ln=True)
          pdf.cell(20, 10, txt="Your order is confirmed", ln=True)
          # Add image
          for file_name in uploaded.keys():
              img = Image.open(file_name)
              pdf.image(file_name, x=155, y=20, w=50)  # Adjust x, y, w as needed

          # Save PDF locally in Colab
          pdf_file_name = "combined_data.pdf"
          pdf.output(pdf_file_name)  # Save the PDF file

          # Copy the PDF to Google Drive
          pdf_path = f"/content/drive/MyDrive/{pdf_file_name}"  # Path in Google Drive
          shutil.copy(pdf_file_name, pdf_path)  # Copy the file
          # Create a shareable link
          shareable_link = make_file_shareable(pdf_file_name)
          if shareable_link:
              print(f"Data and image saved in: {pdf_path}")
              print(f"Shareable link to download the PDF: {shareable_link}")

          # Example usage
          to_email = email
          subject = "Your Shareable Link"
          body = f"Here is your link to download the PDF: {shareable_link}"

          send_email(to_email, subject, body)
chat()









Hi! I am Cyra🤖. Start chatting with me (type 'quit' to stop)!
You: hello
Cyra🤖: Hello!Welcome to Grocery Shop X:) What groceries would you like to order today?
You: what groceries are available
Cyra🤖: Let me know the items and quantities, and I'll prepare your order.
Cyra🤖: [('milk', 1.5), ('bread', 2.0), ('eggs', 3.0), ('cheese', 4.5), ('chicken', 8.0), ('beef', 10.0), ('pork', 7.0), ('fish', 6.0), ('rice', 1.0), ('pasta', 1.5), ('flour', 0.75), ('sugar', 1.2), ('salt', 0.5), ('pepper', 2.5), ('butter', 3.0), ('olive oil', 5.0), ('vegetable oil', 4.0), ('vinegar', 1.0), ('ketchup', 2.0), ('mustard', 1.5), ('mayonnaise', 3.0), ('yogurt', 1.5), ('apples', 0.6), ('bananas', 0.5), ('oranges', 0.8), ('grapes', 2.5), ('strawberries', 3.5), ('blueberries', 4.0), ('raspberries', 5.0), ('watermelon', 4.5), ('cantaloupe', 3.0), ('pineapple', 2.0), ('lemon', 0.5), ('lime', 0.3), ('avocado', 1.0), ('nuts', 8.0), ('almonds', 9.0), ('walnuts', 10.0), ('peanuts', 3.0), ('cashews', 12.0), ('dried fru

Saving WhatsApp Image 2024-11-06 at 2.16.35 PM (1).jpeg to WhatsApp Image 2024-11-06 at 2.16.35 PM (1) (9).jpeg
Data and image saved in: /content/drive/MyDrive/combined_data.pdf
Shareable link to download the PDF: https://drive.google.com/file/d/1--rCfNcLdJNQQ8kq33NUdZvo4BBpmgKi/view?usp=sharing
Email sent successfully!
